In [1]:
import numpy as np
import pandas as pd
import math

In [ ]:
df1 = pd.read_csv('IDF % 2000_2011_2021.csv')
df1 = df1[['Region', 'Country/Territory', '2000', '2011', '2021', 'Type']]
idf = df1.drop(df1.loc[df1['2011']=='-'].index)
idf = idf.drop(idf.loc[idf['2021'] == '-'].index)
idf = idf.drop(columns = '2000')
idf = idf[idf['Type'] == 'Country']

df2 = pd.read_csv('IDF % 2015.csv')
df2 = df2.rename(columns = {'Country/territory': 'Country/Territory'})
final_df = pd.merge(idf, df2, on='Country/Territory')
final_df = final_df.drop(columns = '#')

df3 = pd.read_csv('worldbank 2017.csv')
df3['2017'] = round(df3['Value'],1)
df3 = df3.rename(columns = {'Country Name': 'Country/Territory'})

df4 = pd.read_csv('worldbank 2010_2019.csv')
df4 = df4.rename(columns = {'Diabetes prevalence (% of population ages 20 to 79)': '%'})
df4 = df4.rename(columns = {'Entity': 'Country/Territory'})
df4['%'] = round(df4['%'], 1)
df4_2010 = df4[df4['Year'] == 2010]
df4_2019 = df4[df4['Year'] == 2019]
df4_2010 = df4_2010.rename(columns = {'%': '2010'})
df4_2019 = df4_2019.rename(columns = {'%': '2019'})
df4_final = pd.merge(df4_2019, df4_2010, on='Country/Territory')
df4_final = df4_final[['Country/Territory', '2019', '2010']]

final_df = pd.merge(final_df, df4_final, on='Country/Territory')
final_df = pd.merge(final_df, df3, on='Country/Territory')

In [53]:
cluster_df = pd.read_csv(r"final_countries_cluster.csv")
cluster_df = cluster_df[['Country/Territory', 'cluster']]
cluster_df = cluster_df.rename(columns = {'Country/Territory':'Country'})

macro_df = pd.read_excel(r"macro_indicators_edited.xlsx")
heads = macro_df['Subject Descriptor'].unique()
macro_df2 = macro_df.groupby(['Subject Descriptor', 'Units'])
for i in list(macro_df2):
    col_name = ''.join([i[0][0], "_", i[0][1], "_average"])
    macro_df4 = pd.DataFrame(i[1])
    macro_df4 = macro_df4.dropna(thresh=12)
    macro_df4[col_name] = macro_df4.iloc[:, 4:15].mean(axis=1)
    macro_df5 = macro_df4[['Country', col_name]]
    macro_df5 = macro_df5.reset_index(drop=True)
    macro_df5.set_index('Country', inplace=True)
    try:
        final_macro_df
    except NameError:
        final_macro_df = cluster_df
        final_macro_df = final_macro_df.merge(macro_df5, how="left", on="Country")
    else:
        final_macro_df = final_macro_df.merge(macro_df5, how="left", on="Country")
    
final_macro_df.dropna(thresh=40, inplace=True)
final_macro_df.dropna(axis='columns', thresh=110, inplace=True)

,Country,cluster,"Gross domestic product corresponding to fiscal year, current prices_National currency_average_x","Gross domestic product per capita, constant prices_National currency_average_x","Gross domestic product per capita, constant prices_Purchasing power parity; 2017 international dollar_average_x","Gross domestic product per capita, current prices_National currency_average_x","Gross domestic product per capita, current prices_Purchasing power parity; international dollars_average_x","Gross domestic product per capita, current prices_U.S. dollars_average_x","Gross domestic product, constant prices_National currency_average_x","Gross domestic product, constant prices_Percent change_average_x",...,"Inflation, end of period consumer prices_Percent change_average",Output gap in percent of potential GDP_Percent of potential GDP_average,Population_Persons_average,Six-month London interbank offered rate (LIBOR)_Percent_average,Total investment_Percent of GDP_average,Unemployment rate_Percent of total labor force_average,Volume of Imports of goods_Percent change_average,Volume of exports of goods_Percent change_average,Volume of exports of goods and services_Percent change_average,Volume of imports of goods and services_Percent change_average
0,Afghanistan,6.0,1178.270091,4.093420e+04,2436.314545,4.093385e+04,2367.092727,670.039273,1167.640182,4.164909,...,4.676000,NaN,28.506091,NaN,21.044818,NaN,6.558636,6.359818,-0.353636,5.683727
1,Albania,5.0,1454.438182,2.595020e+05,12370.425455,5.035106e+05,11964.526364,4554.831818,749.657727,2.065091,...,1.827727,NaN,2.889545,NaN,26.692636,14.204091,1.017545,11.164909,2.373273,-0.278182
2,Algeria,5.0,17215.840909,1.719184e+05,11319.478182,4.283464e+05,12352.053636,4544.988182,6884.462727,1.992364,...,4.384091,NaN,40.031818,NaN,44.868818,11.074636,0.101091,-2.320455,-1.582636,-0.726000
3,Andorra,5.0,2.581636,3.471153e+04,56692.730909,3.544135e+04,53733.359091,43333.680000,2.526455,-0.988545,...,0.778909,NaN,0.073000,NaN,NaN,3.634091,NaN,NaN,NaN,NaN
4,Angola,2.0,18082.653636,5.674003e+04,7789.260909,6.528749e+05,7459.172727,4104.394545,1514.165455,1.536364,...,17.128000,NaN,26.831636,NaN,26.123545,NaN,-5.699000,-2.404273,-2.487818,-7.688818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Uruguay,5.0,1571.346182,4.777036e+05,21530.399091,4.518729e+05,21311.001818,16745.486364,1656.537273,2.094364,...,8.185545,NaN,3.466000,NaN,16.824455,7.616727,-3.209000,-1.838125,-4.752500,-3.051500
153,Uzbekistan,3.0,272801.905455,7.010337e+06,6345.157273,8.520333e+06,6712.248182,2161.371818,219322.753636,6.021091,...,12.179455,NaN,31.064000,NaN,31.053818,NaN,8.606273,3.707727,3.904091,8.647636
154,Vanuatu,1.0,83.526909,2.301673e+05,2644.597273,3.086579e+05,2576.143636,2983.479091,61.885364,1.852818,...,2.467909,NaN,0.268909,NaN,29.602727,NaN,NaN,NaN,NaN,NaN
155,Zambia,3.0,203.332091,7.552190e+03,3303.620000,1.213992e+04,3380.108182,1474.835455,123.230909,4.170273,...,10.081455,NaN,16.287091,NaN,36.480455,NaN,6.510091,5.286727,5.286727,6.510091


In [ ]:
edu = pd.read_csv(r"Education index.csv")
final_edu = edu.dropna(axis = 'columns')
final_edu['Education Index_average'] = final_edu.iloc[:, 1:].mean(axis=1)
final_macro_df = pd.merge(final_macro_df, final_edu, how='left', on=['Country'])